# 从零开始实现字节对编码（BPE）分词器

- 这是一个独立的notebook，从零开始实现流行的字节对编码（BPE）分词算法，该算法用于GPT-2到GPT-4、Llama 3等模型，用于教育目的
- 有关分词目的的更多详细信息，请参考[第二章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这里的代码是解释BPE算法的补充材料
- OpenAI为训练原始GPT模型而实现的原始BPE分词器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到
- BPE算法最初在1994年由Philip Gage描述："[数据压缩的新算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"
- 如今大多数项目，包括Llama 3，都使用OpenAI的开源[tiktoken库](https://github.com/openai/tiktoken)，因为它具有计算性能；它允许加载预训练的GPT-2和GPT-4分词器，例如（Llama 3模型也是使用GPT-4分词器训练的）
- 上述实现与本notebook中我的实现之间的区别，除了它是教育目的外，还包括训练分词器的功能（用于教育目的）
- 还有一个名为[minBPE](https://github.com/karpathy/minbpe)的实现，支持训练，可能性能更好（我这里的实现专注于教育目的）；与`minbpe`相比，我的实现还允许加载原始OpenAI分词器词汇表和BPE"合并"（此外，Hugging Face分词器也能够训练和加载各种分词器；有关更多信息，请参阅读者在尼泊尔语上训练BPE分词器的[GitHub讨论](https://github.com/rasbt/LLMs-from-scratch/discussions/485)）

&nbsp;
# 1. 字节对编码（BPE）背后的主要思想

- BPE中的主要思想是将文本转换为整数表示（token ID）用于LLM训练（参见[第二章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">

&nbsp;
## 1.1 位和字节

- 在了解BPE算法之前，让我们介绍字节的概念
- 考虑将文本转换为字节数组（毕竟BPE代表"字节"对编码）：

In [39]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


- 当我们对`bytearray`对象调用`list()`时，每个字节都被视为一个单独的元素，结果是对应于字节值的整数列表：

In [40]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- 这将是将文本转换为LLM嵌入层所需的token ID表示的有效方法
- 然而，这种方法的缺点是它为每个字符创建一个ID（对于短文本来说ID太多了！）
- 也就是说，这意味着对于17个字符的输入文本，我们必须使用17个token ID作为LLM的输入：

In [41]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果您之前使用过LLM，您可能知道BPE分词器有一个词汇表，其中我们有整个单词或子词的token ID，而不是每个字符的ID
- 例如，GPT-2分词器将相同的文本（"This is some text"）分词为只有4个而不是17个token：`1212, 318, 617, 2420`
- 您可以使用交互式[tiktoken应用](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken库](https://github.com/openai/tiktoken)来验证这一点：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```

- 由于一个字节由8位组成，单个字节可以表示2<sup>8</sup> = 256个可能的值，范围从0到255
- 您可以通过执行代码`bytearray(range(0, 257))`来确认这一点，这将警告您`ValueError: byte must be in range(0, 256)`）
- BPE分词器通常使用这256个值作为其前256个单字符token；可以通过运行以下代码来直观地检查这一点：

```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上面，注意条目256和257不是单字符值而是双字符值（空格+字母），这是原始GPT-2 BPE分词器的一个小缺点（这在GPT-4分词器中得到了改进）

&nbsp;
## 1.2 构建词汇表

- BPE分词算法的目标是构建一个常用子词的词汇表，如`298: ent`（可以在*entangle, entertain, enter, entrance, entity, ...*等词中找到），甚至是完整的单词，如

```
318: is
617: some
1212: This
2420: text
```

- BPE算法最初在1994年由Philip Gage描述："[数据压缩的新算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"
- 在我们进入实际代码实现之前，今天用于LLM分词器的形式可以总结为以下部分中描述的内容。

&nbsp;
## 1.3 BPE算法概述

**1. 识别频繁对**
- 在每次迭代中，扫描文本以找到最常出现的字节（或字符）对

**2. 替换和记录**

- 将该对替换为新的占位符ID（一个尚未使用的ID，例如，如果我们从0...255开始，第一个占位符将是256）
- 在查找表中记录此映射
- 查找表的大小是一个超参数，也称为"词汇表大小"（对于GPT-2，这是50,257）

**3. 重复直到无增益**

- 继续重复步骤1和2，不断合并最频繁的对
- 当无法进一步压缩时停止（例如，没有对出现超过一次）

**解压缩（解码）**

- 要恢复原始文本，通过使用查找表将每个ID替换为其对应的对，反向进行此过程



&nbsp;
&nbsp;
## 1.4 BPE算法示例

### 1.4.1 编码部分的具体示例（第1.3节中的步骤1和2）

- 假设我们有文本（训练数据集）`the cat in the hat`，我们想从中构建BPE分词器的词汇表

**迭代1**

1. 识别频繁对
  - 在此文本中，"th"出现两次（在开头和第二个"e"之前）

2. 替换和记录
  - 将"th"替换为尚未使用的新token ID，例如256
  - 新文本是：`<256>e cat in <256>e hat`
  - 新词汇表是

```
  0: ...
  ...
  256: "th"
```

**迭代2**

1. **识别频繁对**  
   - 在文本`<256>e cat in <256>e hat`中，对`<256>e`出现两次

2. **替换和记录**  
   - 将`<256>e`替换为尚未使用的新token ID，例如`257`。  
   - 新文本是：
     ```
     <257> cat in <257> hat
     ```
   - 更新的词汇表是：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**迭代3**

1. **识别频繁对**  
   - 在文本`<257> cat in <257> hat`中，对`<257> `出现两次（一次在开头，一次在"hat"之前）。

2. **替换和记录**  
   - 将`<257> `替换为尚未使用的新token ID，例如`258`。  
   - 新文本是：
     ```
     <258>cat in <258>hat
     ```
   - 更新的词汇表是：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- 以此类推

&nbsp;
### 1.4.2 解码部分的具体示例（第1.3节中的步骤3）

- 要恢复原始文本，我们通过按引入的相反顺序将每个token ID替换为其对应的对来反向进行此过程
- 从最终压缩文本开始：`<258>cat in <258>hat`
- 替换`<258>` → `<257> `：`<257> cat in <257> hat`  
- 替换`<257>` → `<256>e`：`<256>e cat in <256>e hat`
- 替换`<256>` → "th"：`the cat in the hat`


&nbsp;
## 2. 简单的BPE实现

- 下面是上述算法的实现，作为一个Python类，模仿`tiktoken` Python用户界面
- 注意，上面的编码部分描述了通过`train()`进行的原始训练步骤；但是，`encode()`方法的工作方式类似（尽管由于特殊token处理看起来更复杂一些）：

1. 将输入文本分割为单个字节
2. 重复查找和替换（合并）相邻的token（对），当它们与学习的BPE合并中的任何对匹配时（从最高到最低"排名"，即按学习的顺序）
3. 继续合并直到无法应用更多合并
4. 最终的token ID列表是编码输出

In [ ]:
from collections import Counter, deque
from functools import lru_cache
import json


class BPETokenizerSimple:
    def __init__(self):
        # 映射token_id到token_str（例如，{11246: "some"}）
        self.vocab = {}
        # 映射token_str到token_id（例如，{"some": 11246}）
        self.inverse_vocab = {}
        # BPE合并字典：{(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # 对于官方OpenAI GPT-2合并，使用排名字典：
        # 形式为{(string_A, string_B): rank}，其中较低排名=较高优先级
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        从零开始训练BPE分词器。

        Args:
            text (str): 训练文本。
            vocab_size (int): 期望的词汇表大小。
            allowed_special (set): 要包含的特殊token集合。
        """

        # 预处理：将空格替换为"Ġ"
        # 注意Ġ是GPT-2 BPE实现的一个特殊性
        # 例如，"Hello world"可能被分词为["Hello", "Ġworld"]
        # （GPT-4 BPE会将其分词为["Hello", " world"]）
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # 用唯一字符初始化词汇表，如果存在则包括"Ġ"
        # 从前256个ASCII字符开始
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # 添加允许的特殊token
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # 将processed_text分词为token ID
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE步骤1-3：重复查找和替换频繁对
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # 用合并的token构建词汇表
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        从OpenAI的GPT-2文件加载预训练词汇表和BPE合并。

        Args:
            vocab_path (str): 词汇表文件路径（GPT-2称之为'encoder.json'）。
            bpe_merges_path (str): bpe_merges文件路径（GPT-2称之为'vocab.bpe'）。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # 将加载的词汇表转换为正确格式
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # 处理换行符而不添加新token
        if "\n" not in self.inverse_vocab:
            # 使用现有token ID作为'\n'的占位符
            # 优先使用"<|endoftext|>"如果可用
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # 如果没有可用的后备token，抛出错误
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # 加载GPT-2合并并存储它们与分配的"排名"
        self.bpe_ranks = {}  # 重置排名
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # 如果token1或token2不在词汇表中，跳过
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} as one token is not in the vocabulary.")

    def encode(self, text, allowed_special=None):
        """
        将输入文本编码为token ID列表，具有tiktoken风格的特殊token处理。
    
        Args:
            text (str): 要编码的输入文本。
            allowed_special (set or None): 允许通过的特殊token。如果为None，则禁用特殊处理。
    
        Returns:
            token ID列表。
        """
        import re
    
        token_ids = []
    
        # 如果启用了特殊token处理
        if allowed_special is not None and len(allowed_special) > 0:
            # 构建正则表达式以匹配允许的特殊token
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )
    
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # 编码前缀而不进行特殊处理
    
                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
    
            text = text[last_index:]  # 剩余部分正常处理
    
            # 检查剩余部分中是否有不允许的特殊token
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")
    
        # 如果没有特殊token，或特殊token分割后的剩余文本：
        tokens = []
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:
                    tokens.append("Ġ" + word)
                elif j == 0:
                    tokens.append(word)
                else:
                    tokens.append("Ġ" + word)
    
        for token in tokens:
            if token in self.inverse_vocab:
                token_ids.append(self.inverse_vocab[token])
            else:
                token_ids.extend(self.tokenize_with_bpe(token))
    
        return token_ids

    def tokenize_with_bpe(self, token):
        """
        使用BPE合并对单个token进行分词。

        Args:
            token (str): 要分词的token。

        Returns:
            List[int]: 应用BPE后的token ID列表。
        """
        # 将token分词为单个字符（作为初始token ID）
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # 如果我们没有加载OpenAI的GPT-2合并，使用我的方法
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # 为了教育目的取消注释：
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # 跳过下一个token，因为它被合并了
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # 否则，使用排名进行GPT-2风格的合并：
        # 1) 将token_ids转换回每个ID的字符串"符号"
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # 重复合并所有最低排名对的出现
        while True:
            # 收集所有相邻对
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # 找到最佳（最低）排名的对
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # 如果没有有效的排名对存在，我们就完成了
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # 合并该对的所有出现
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # 如果我们在位置i看到(first, second)，合并它们
                if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                    new_symbols.append(first + second)  # 合并的符号
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # 最后，将合并的符号转换回ID
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        将token ID列表解码回字符串。

        Args:
            token_ids (List[int]): 要解码的token ID列表。

        Returns:
            str: 解码的字符串。
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # 如果换行前没有空格，添加空格
                decoded_string += token
            elif token.startswith("Ġ"):
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        将词汇表和BPE合并保存到JSON文件。

        Args:
            vocab_path (str): 保存词汇表的路径。
            bpe_merges_path (str): 保存BPE合并的路径。
        """
        # 保存词汇表
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump(self.vocab, file, ensure_ascii=False, indent=2)

        # 将BPE合并保存为字典列表
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        从JSON文件加载词汇表和BPE合并。

        Args:
            vocab_path (str): 词汇表文件路径。
            bpe_merges_path (str): BPE合并文件路径。
        """
        # 加载词汇表
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # 加载BPE合并
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge["new_id"]
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # 移除对的第2个token，第1个已经被移除
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

- 上面的`BPETokenizerSimple`类中有很多代码，详细讨论超出了本notebook的范围，但下一节提供了使用概述，以便更好地理解类方法

## 3. BPE实现演练

- 在实践中，我强烈建议使用[tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现专注于可读性和教育目的，而不是性能
- 但是，使用方式或多或少与tiktoken类似，除了tiktoken没有训练方法
- 让我们通过查看下面的一些示例来了解我上面的`BPETokenizerSimple` Python代码是如何工作的（详细的代码讨论超出了本notebook的范围）

### 3.1 训练、编码和解码

- 首先，让我们考虑一些示例文本作为我们的训练数据集：

In [71]:
import os
import urllib.request

def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        with urllib.request.urlopen(url) as response, open(target_path, "wb") as out_file:
            out_file.write(response.read())
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")
    return target_path

verdict_path = download_file_if_absent(
    url=(
         "https://raw.githubusercontent.com/rasbt/"
         "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
         "the-verdict.txt"
    ),
    filename="the-verdict.txt",
    search_dirs=["ch02/01_main-chapter-code/", "../01_main-chapter-code/", "."]
)

with open(verdict_path, "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

the-verdict.txt already exists in ../01_main-chapter-code/the-verdict.txt


- 接下来，让我们初始化并训练BPE分词器，词汇表大小为1,000
- 注意，由于前面讨论的字节值，词汇表大小已经是256，所以我们只"学习"744个词汇表条目（如果我们考虑`<|endoftext|>`特殊token和`Ġ`空格token；所以，精确地说是742个）
- 为了比较，GPT-2词汇表有50,257个token，GPT-4词汇表有100,256个token（tiktoken中的`cl100k_base`），GPT-4o使用199,997个token（tiktoken中的`o200k_base`）；与我们上面的简单示例文本相比，它们都有更大的训练集

In [46]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

- 您可能想要检查词汇表内容（但请注意它会创建一个很长的列表）

In [47]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


- 这个词汇表是通过合并742次创建的（`= 1000 - len(range(0, 256)) - len(special_tokens) - "Ġ" = 1000 - 256 - 1 - 1 = 742`）

In [48]:
print(len(tokenizer.bpe_merges))

742


- 这意味着前256个条目是单字符token

- 接下来，让我们使用创建的合并通过`encode`方法来编码一些文本：

In [49]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [50]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 60, 124, 271, 683, 102, 116, 461, 116, 124, 62]


In [51]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


In [52]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 56
Number of token IDs: 21


- 从上面的长度可以看出，一个42字符的句子被编码为20个token ID，与基于字符字节的编码相比，有效地将输入长度减少了一半

- 注意，词汇表本身在`decode()`方法中使用，这允许我们将token ID映射回文本：

In [53]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


In [54]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|>


- Iterating over each token ID can give us a better understanding of how the token IDs are decoded via the vocabulary:

In [55]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .
257 -> <|endoftext|>


- 正如我们所看到的，大多数token ID代表2字符子词；这是因为训练数据文本很短，没有那么多重复的单词，而且因为我们使用了相对较小的词汇表大小

- 作为总结，调用`decode(encode())`应该能够重现任意输入文本：

In [56]:
tokenizer.decode(
    tokenizer.encode("This is some text.")
)

'This is some text.'

In [57]:
tokenizer.decode(
    tokenizer.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

### 3.2 保存和加载分词器

- 接下来，让我们看看如何保存训练好的分词器以供以后重用：

In [58]:
# Save trained tokenizer
tokenizer.save_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [59]:
# Load tokenizer
tokenizer2 = BPETokenizerSimple()
tokenizer2.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

- The loaded tokenizer should be able to produce the same results as before:

In [60]:
print(tokenizer2.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|>


In [61]:
tokenizer2.decode(
    tokenizer2.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

&nbsp;
### 3.3 从OpenAI加载原始GPT-2 BPE分词器

- 最后，让我们加载OpenAI的GPT-2分词器文件

In [72]:
# Download files if not already present in this directory

# Define the directories to search and the files to download
search_directories = ["ch02/02_bonus_bytepair-encoder/gpt2_model/", "../02_bonus_bytepair-encoder/gpt2_model/", "."]

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

# Ensure directories exist and download files if needed
paths = {}
for url, filename in files_to_download.items():
    paths[filename] = download_file_if_absent(url, filename, search_directories)

vocab.bpe already exists in ../02_bonus_bytepair-encoder/gpt2_model/vocab.bpe
encoder.json already exists in ../02_bonus_bytepair-encoder/gpt2_model/encoder.json


- 接下来，我们通过`load_vocab_and_merges_from_openai`方法加载文件：

In [23]:
tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=paths["encoder.json"], bpe_merges_path=paths["vocab.bpe"]
)

- 词汇表大小应该是`50257`，我们可以通过下面的代码确认：

In [24]:
len(tokenizer_gpt2.vocab)

50257

- 我们现在可以通过我们的`BPETokenizerSimple`对象使用GPT-2分词器：

In [25]:
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print(token_ids)

[1212, 318, 617, 2420]


In [26]:
print(tokenizer_gpt2.decode(token_ids))

This is some text


- 您可以使用交互式[tiktoken应用](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken库](https://github.com/openai/tiktoken)来验证这产生了正确的token：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```


&nbsp;
# 4. 结论

- 就是这样！这就是BPE的工作原理，包括创建新分词器的训练方法或从原始OpenAI GPT-2模型加载GPT-2分词器词汇表和合并
- 我希望您发现这个简短的教程对教育目的有用；如果您有任何问题，请随时在[这里](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a)开启新的讨论
- 有关与其他分词器实现的性能比较，请参见[此notebook](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb)